In [12]:
import random
import json
import pickle
import numpy as np
import nltk
import tensorflow as tf

In [7]:
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

In [8]:


lemmatizer = WordNetLemmatizer()
intents = json.loads(open(r"C:\Users\akash s ranjan\Downloads\bankproject\customer_service\intents.json", encoding="utf-8").read())


In [9]:
words = pickle.load(open('C:/Users/akash s ranjan/Downloads/bankproject/customer_service/words.pkl', 'rb'))
classes = pickle.load(open('C:/Users/akash s ranjan/Downloads/bankproject/customer_service/classes.pkl', 'rb'))

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the number of classes based on your intents
num_classes = len(classes)

# Recreate the model architecture
model = Sequential([
    Dense(128, input_shape=(121,), activation='relu'),  # Removed None from input shape
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Load the weights
model.load_weights('C:/Users/akash s ranjan/Downloads/bankproject/customer_service/chatbot_model.h5')  # Load only the weights, not the entire model

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [22]:

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [23]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

In [24]:
import numpy as np

def predict_class(message):
    bow = bag_of_words(message)

    res = model.predict(np.array([bow]).reshape(1, -1))[0]

    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [25]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice (i['responses'])
            break
    return result

In [26]:
print(model.summary())


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │        15,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 13)             │           845 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,717 (96.55 KB)

 Trainable params: 24,717 (96.55 KB)

 Non-trainable params: 0 (0.00 B)

None


In [27]:
print("The Chatbot is running!")

while True:
    message = input("")
    ints = predict_class (message)
    res = get_response (ints, intents)
    print (res)

The Chatbot is running!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Welcome
